# Trilateration
### Given the distance from three points, find the location of the 4th

In [7]:
import numpy as np
from scipy.optimize import least_squares
import folium
from IPython.display import display

In [9]:
#Coordinates and their distances from Washington University 
hc = [38.75404, -90.4816] #Hollywood Casino - 11.60 miles away
bg = [38.61276, -90.25937] #Botanical Garden - 3.73 miles away
mt = [38.57304, -90.46289] #Museum of Transportation - 9.70 miles away

home = [38.57109, -90.29808] #12
work = [38.85720, -90.06833] #35
gs = [38.60840, -90.21133] #16

locations = np.array([home,work,gs])

#Distances to the each location in miles
#distances = np.array([11.6, 3.73, 9.7]) #10th's and 100th's
distances = np.array([12, 35, 16]) #Lower resolution

def haversine(coord1, coord2): #Calculate the great-circle distance between two points on Earth in decimal degrees.
    lat1, lon1 = np.radians(coord1)
    lat2, lon2 = np.radians(coord2)
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))

    return 3958.8 * c  #Earth's radius in miles

def residuals(x, locations, distances): #Calculate residuals for the optimization.
    return np.array([haversine(x, locations[i]) - distances[i] for i in range(len(locations))])

#Initial guess (mean of tower coordinates)
initial_guess = np.mean(locations, axis=0)

#Perform least squares optimization to find the point that minimizes the residuals
result = least_squares(residuals, initial_guess, args=(locations, distances))

#Extract the optimized coordinates
location = result.x
print(f"Estimated location: {location}")

Estimated location: [ 38.40170624 -90.3480774 ]


In [11]:
#Create a map centered around the estimated location
m = folium.Map(location=location, zoom_start=6)

#Add markers for the locations and estimated location
for i, (lat, lon) in enumerate(locations):
    folium.Marker([lat, lon], popup=f"Tower {i+1}").add_to(m)

folium.Marker(location, popup="Estimated Location", icon=folium.Icon(color='red')).add_to(m)

#Show the map
display(m)

#Save the map to an HTML file
#m.save("trilateration_map.html")